In [9]:
import os, sys, pickle
if '..' not in sys.path:
     sys.path.insert(0, '..')

import numpy as np
import pandas as pd

from PIL import Image
from collections import OrderedDict

import torch
import torchvision.models as models
import torchvision.transforms as T

import utils_models as mut


if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print(f'you are not using a gpu (or your cuda install is messed up). fix it.')

NVIDIA A100-SXM4-40GB MIG 3g.20gb


In [2]:
# load pretrained alexnet (imagenet)
model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [3]:
# load in NSD and localizer images (total: 1072 images)
IMG_DIR = '../../datasets/NNN/NSD1000_LOC'
IMG_PATHS = [os.path.join(IMG_DIR, img) for img in os.listdir(IMG_DIR) if '.tsv' not in img]
print(f'Total number of images found: {len(IMG_PATHS)}')

img_tensor = mut.load_images(IMG_PATHS)
acts = mut.get_activations(model, img_tensor)

Total number of images found: 1072


In [7]:
# FOR REFERENCE, ALEXNET LAYERS + ACTUAL ACTs
for name, module in model.named_modules():
    if '.' in name:
        print(f'{name:<15} | {module.__class__.__name__:>15}')
        
print('\n\n', '='*50, '\n\n')
        
for layer, out in acts.items():
    print(f'{layer:<15} | {str(out.shape):>35}')
    
with open('alexnet_acts.pkl', 'wb') as f:
    pickle.dump(acts, f)

features.0      |          Conv2d
features.1      |            ReLU
features.2      |       MaxPool2d
features.3      |          Conv2d
features.4      |            ReLU
features.5      |       MaxPool2d
features.6      |          Conv2d
features.7      |            ReLU
features.8      |          Conv2d
features.9      |            ReLU
features.10     |          Conv2d
features.11     |            ReLU
features.12     |       MaxPool2d
classifier.0    |         Dropout
classifier.1    |          Linear
classifier.2    |            ReLU
classifier.3    |         Dropout
classifier.4    |          Linear
classifier.5    |            ReLU
classifier.6    |          Linear




features.0      |      torch.Size([1072, 64, 55, 55])
features.1      |      torch.Size([1072, 64, 55, 55])
features.2      |      torch.Size([1072, 64, 27, 27])
features.3      |     torch.Size([1072, 192, 27, 27])
features.4      |     torch.Size([1072, 192, 27, 27])
features.5      |     torch.Size([1072, 192, 1